## 3.0 LLMs and Chat Models
- 단순 텍스트를 predict 해보기
- OpenAI 사용시, text-davinci-003 모델을 BaseModel로 사용하지만, 지금은(24.04.09 기준) depreceated 됨

In [9]:
from langchain.llms import OpenAI #BaseModel is text-davinci-003 which is triple expensive than gpt-3.5-turbo
from langchain.chat_models import ChatOpenAI #BaseModel is gpt-3.5-turbo

llm = OpenAI(model_name="gpt-3.5-turbo-1106")
chat = ChatOpenAI()

a = llm.predict("How many planets are there?")
b = chat.predict("How many planets  are there?")

a,b

d:\FULLSTACK-GPT\env\Lib\site-packages\langchain\llms\openai.py:216: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
d:\FULLSTACK-GPT\env\Lib\site-packages\langchain\llms\openai.py:811: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


('There are eight officially recognized planets in our solar system: Mercury, Venus, Earth, Mars, Jupiter, Saturn, Uranus, and Neptune. However, there is ongoing debate about whether or not Pluto should be considered a planet.',
 'There are eight planets in our solar system: Mercury, Venus, Earth, Mars, Jupiter, Saturn, Uranus, and Neptune.')

## 3.1 Predict Messages
- 모델 constructor를 이용해서 변수를 설정할 수 있음

In [8]:
from langchain.chat_models import ChatOpenAI

chat = ChatOpenAI(
    #max_tokens=1000, #최대 토큰값 
    temperature=0.1 #0~1값 입력
)

- string을 단순 predict하는 것과 메세지 List들을 predict 하는것은 차이가 존재함.
- SystemMessage: LLM에 설정들을 제공하기 위한 메세지. 일종의 설정들을 AI에게 넘겨주기 위한 메세지
- AIMessage: AI에 의해 보내지는 메세지. AI가 답변을 한것처럼 설정가능함.
- HumanMessage: 사용자 메세지

In [11]:
from langchain.schema import HumanMessage, AIMessage, SystemMessage 

messages = [ #context 설정
    SystemMessage(content="당신은 지리 전문가입니다. 그리고 당신은 이탈리아어로만 답변합니다.") #LLM에 설정들을 제공하기 위한 메세지
    ,
    AIMessage(content="안녕하세요 제 이름은 Heidi입니다."), #AI에 의해 보내지는 메세지
    HumanMessage(content="멕시코와 태국 사이의 거리는 어떻게 되나요?그리고 당신의 이름은 무엇인가요?") #사용자 메세지
]

chat.predict_messages(messages)

AIMessage(content='안녕하세요! 멕시코와 태국 사이의 거리는 대략 16,000km 정도입니다. 제 이름은 헤이디입니다. 어떤 도움이 필요하신가요?')

## 3.2 Prompt Templates
- ChatPromptTemplate: message로부터 template 생성
- PromptTemplate: string을 이용해서 template 생성
- {중괄호기법}: template에 일종의 placeholder 생성
- template.format을 통해 placeholder의 값 선언. template 상위계층에 검증계층(validation layer) 존재하여 자동으로 유효성검사 실행됨.

In [13]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate

chat = ChatOpenAI(temperature=0.1)

template = PromptTemplate.from_template("{country_a}와 {country_b} 사이의 거리는 몇인가요?")

prompt = template.format(country_a="Mexico", country_b="Thailand")

chat.predict(prompt)


'Mexico와 Thailand 사이의 직선 거리는 약 16,000km 입니다.'

In [22]:
template = ChatPromptTemplate.from_messages([ 
    ("system","you are a geography expert. and you only reply in {language}."), ##한국어로 설정메세지 입력하니 잘 안되서. 영어로 변경함
    ("ai","안녕하세요 제 이름은 {name}입니다."), 
    ("human","{country_a}와 {country_b} 사이의 거리는 어떻게 되나요?그리고 당신의 이름은 무엇인가요?")
])

prompt = template.format_messages(
    language="Greek",
    name="Kitty",
    country_a="대한민국",
    country_b="미국"
)

chat.predict_messages(prompt)

AIMessage(content='Γεια σας! Η απόσταση μεταξύ Νότιας Κορέας και Ηνωμένων Πολιτειών εξαρτάται από την συγκεκριμένη πόλη στην οποία αναφερόμαστε. Για παράδειγμα, η απόσταση μεταξύ της Σεούλ και της Ουάσιγκτον DC είναι περίπου 11.000 χιλιόμετρα. Το όνομά μου είναι Kitty. Πώς μπορώ να βοηθήσω;')

## 3.3 OutputParser and LCEL
- LCEL(Langchain Expression Language)
- Output Parser: LLM의 output을 구조별로 parse(파싱) 할 수 있게함. 
- chain: 모든 요소를 합쳐주는 역할. template, chatmodel, parser를 코드 한 줄로 연결해줌

In [24]:
from langchain.schema import BaseOutputParser

class CommaOutputParser(BaseOutputParser):
    def parse(self, text): #메소드
        items = text.strip().split(",") #text의 앞뒤 공백 제거 후, 콤마로 자르기
        return list(map(str.strip, items)) #item의 각 요소에 앞뒤 공백 제거 후, list로 리턴

p = CommaOutputParser()
p.parse("Hello ,how , are, you")
    
        

['Hello', 'how', 'are', 'you']

In [29]:
template = ChatPromptTemplate.from_messages([
    ("system","You are a list generation machine. Everything you are asked will be answered with a comma separated list of max {max_items} in lower case. DO NOT reply with anything else."),
    ("human","{question}"),
])

prompt = template.format_messages(
    max_items=10,
    question="What are the planets?"
)

chain = template | chat | CommaOutputParser() #chain 생성("|"기호 사용), 코드를 간소화시킬수있음

chain.invoke({
    "max_items":5,
    "question": "What are the pokemons?"
})

['pikachu', 'charmander', 'bulbasaur', 'squirtle', 'jigglypuff']

## 3.4 Chaining Chains
- chain은 각 component를 입력받아서, 옆의 component에게 결과값을 전달한다.
- The input type and output type varies by component:     


|   Component  |                       Input Type                      |      Output Type      |
|:------------:|:-----------------------------------------------------:|:---------------------:|
| Prompt       | Dictionary                                            | PromptValue           |
| ChatModel    | Single string, list of chat messages or a PromptValue | ChatMessage           |
| LLM          | Single string, list of chat messages or a PromptValue | String                |
| OutputParser | The output of an LLM or ChatModel                     | Depends on the parser |
| Retriever    | Single string                                         | List of Documents     |
| Tool         | Single string or dictionary, depending on the tool    | Depends on the tool   |

In [5]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
)

chef_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a world-class international chef. You create easy to follow recipies for any type of cuisine with easy to find ingredients."),
    ("human","I want to cook {cuisine} food.")
])

chef_chain = chef_prompt | chat

veg_chef_prompt = ChatPromptTemplate.from_messages([
    ("system","You are a vegetarian chef specialized on making tranditional recipies vegetaria. You find alternative ingredients and explain ther preparation. You don't radically modify the recipe. If there is no alternative for a food just say you don't know how to replace it."),
    ("human","{recipe}")
])

veg_chain = veg_chef_prompt | chat 

final_chain = {"recipe": chef_chain} | veg_chain #chef_chain의 output을 recipe로 정의하여 veg_chain의 placeholder로 전달함

final_chain.invoke({
    "cuisine":"indian"
})


Great choice! How about trying to make a classic Indian dish like Chicken Tikka Masala? It's a flavorful and aromatic dish that is sure to impress. Here's a simple recipe for you to try:

Ingredients:
- 1 lb boneless, skinless chicken breasts, cut into bite-sized pieces
- 1 cup plain yogurt
- 2 tablespoons lemon juice
- 2 teaspoons ground cumin
- 2 teaspoons paprika
- 1 teaspoon ground turmeric
- 1 teaspoon garam masala
- 1 teaspoon ground coriander
- 1 teaspoon chili powder (adjust to taste)
- 2 tablespoons vegetable oil
- 1 onion, finely chopped
- 3 cloves garlic, minced
- 1-inch piece of ginger, grated
- 1 can (14 oz) crushed tomatoes
- 1 cup heavy cream
- Salt and pepper to taste
- Fresh cilantro, chopped (for garnish)

Instructions:
1. In a bowl, mix together the yogurt, lemon juice, cumin, paprika, turmeric, garam masala, coriander, and chili powder. Add the chicken pieces and coat them well with the marinade. Cover and refrigerate for at least 1 hour, or overnight for best resul

AIMessageChunk(content='대체재료:\n\n- 대체재료: 대체재료: 대체재료: 대체재료: 대체재료: 대체재료: 대체재료: 대체재료: 대체재료: 대체재료: 대체재료: 대체재료: 대체재료: 대체재료: 대체재료: 대체재료: 대체재료: 대체재료: 대체재료: 대체재료: 대체재료: 대체재료: 대체재료: 대체재료: 대체재료: 대체재료: 대체재료: 대체재료: 대체재료: 대체재료: 대체재료: 대체재료: 대체재료: 대체재료: 대체재료: 대체재료: 대체재료: 대체재료: 대체재료: 대체재료: 대체재료: 대체재료: 대체재료: 대체재료: 대체재료: 대체재료: 대체재료: 대체재료: 대체재료: 대체재료: 대체재료: 대체재료: 대체재료: 대체재료: 대체재료: 대체재료: 대체재료: 대체재료: 대체재료: 대체재료: 대체재료: 대체재료: 대체재료: 대체재료: 대체재료: 대체재료: 대체재료: 대체재료: 대체재료: 대체재료: 대체재료: 대체재료: 대체재료: 대체재료: 대체재료: 대체재료: 대체재료: 대체재료: 대체재료: 대체재료: 대체재료: 대체재료: 대체재료: 대체재료: 대체재료: 대체재료: 대체재료: 대체재료: 대체재료: 대체재료: 대체재료: 대체재료: 대체재료: 대체재료: 대체재료: 대체재료: 대체재료: 대체재료: 대체재료: 대체재료: 대체재료: 대체재료: 대체재료: 대체재료: 대체재료: 대체재료: 대체재료: 대체재료: 대체재료: 대체재료: 대체재료: 대체재료: 대체재료: 대체재료: 대체재료: 대체재료: 대체재료: 대체재료: 대체재료: 대체재료: 대체재료: 대체재료: 대체재료: 대체재료: 대체재료: 대체재료: 대체재료: 대체재료: 대체재료: 대체재료: 대체재료: 대체재료: 대체재료: 대체재료: 대체재료: 대체재료: 대체재료: 대체재료: 대체재료: 대체재료: 대체재료: 대체재료: 대체재료: 대체재료: 대체재료: 대체재료: 대체재료: 대체재료: 대체재료: 대체재료: 대체재료: 대체재료: 대체재료: 대체재료: 대체재료: 대체재료: 대체재료: 대체재료: 대체재료: 대체재료: 대체재료:

## 4.0 Introduction
- Model I/O: Prompts, Chat models, LLMs
- Retrieval: Document loaders, Text splitters, Embedding models, Vectorstores, Retrievers
    - 외부 데이터로 접근하여 이를 모델에 어떻게 제공하는 것에 관한 것
- Composition: Tools, Agents, Chains
    - Agents: AI가 독립적으로 동작하게끔 도와줌. chains이 필요한 tool들을 선택하여 사용할 수 있음
- Additional: Memory, Callbacks
    - Memory: 챗봇에 데이터 저장
    - Callbacks: 답변을 제공하기 전에 모델이 무엇을 하고 있는지 알 수 있도록 하는 것

## 4.1 FewShotPromptTemplate
- FewShot: 모델에게 예제를 주는것. 더 나은 대답을 할 수 있도록하는 예제를 제시. Prompt를 사용해서 어떻게 대답해야 하는지 알려주는것보다 나음.
- prompt 로 전달하는 것보다 원하는 것을 예제로 보여주는것이 성공적.

In [3]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.prompts.few_shot import FewShotPromptTemplate 
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler()
    ]
)
""" #아래와 동일코드 
t = PromptTemplate(
    template="What is the captital of {country}",
    input_variables=["country"],
)
"""
#t = PromptTemplate.from_template("What is the captial of {country}")

#t.format(country="France")

examples = [
    {
        "question": "What do you know about France?",
        "answer": """ 
         Here is what I know:
         Capital: Paris
         Language: French
         Food: Wine and Cheese
         Currency: Euro
         """,
         },
         {
         "question": "What do you know about Italy?",
         "answer": """
         I know this:
         Capital: Rome
         Language: Italian
         Food: Pizza and Pasta
         Currency: Euro
         """,
         },
         {
         "question": "What do you know about Greece?",
         "answer": """
         I know this:
         Capital: Athens
         Language: Greek
         Food: Souvlaki and Feta Cheese
         Currency: Euro
         """,
    },
]


example_prompt = PromptTemplate.from_template("Human: {question}\nAI:{answer}")

prompt = FewShotPromptTemplate(#리스트에 있는 모든 example들을 가지고 형식화
    example_prompt=example_prompt,
    examples=examples,
    suffix="Human: What do you know about {country}?",
    input_variables=["country"]
)

chain = prompt | chat

chain.invoke({
    "country":"Germany"
})


AI:
         I know this:
         Capital: Berlin
         Language: German
         Food: Bratwurst and Sauerkraut
         Currency: Euro

AIMessageChunk(content='AI:\n         I know this:\n         Capital: Berlin\n         Language: German\n         Food: Bratwurst and Sauerkraut\n         Currency: Euro')

Connection error caused failure to post https://api.smith.langchain.com/runs  in LangSmith API. Please confirm your LANGCHAIN_ENDPOINT. SSLError(MaxRetryError("HTTPSConnectionPool(host='api.smith.langchain.com', port=443): Max retries exceeded with url: /runs (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: self-signed certificate in certificate chain (_ssl.c:1006)')))"))
Connection error caused failure to patch https://api.smith.langchain.com/runs/655e582b-e0e8-4c53-9ada-6560b007c3c1  in LangSmith API. Please confirm your LANGCHAIN_ENDPOINT. SSLError(MaxRetryError("HTTPSConnectionPool(host='api.smith.langchain.com', port=443): Max retries exceeded with url: /runs/655e582b-e0e8-4c53-9ada-6560b007c3c1 (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: self-signed certificate in certificate chain (_ssl.c:1006)')))"))


## 4.2 FewShotChatMessagePromptTemplate

In [5]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate #FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import ChatPromptTemplate

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler()
    ]
)

examples = [
    {
        "country": "France",
        "answer": """ 
         Here is what I know:
         Capital: Paris
         Language: French
         Food: Wine and Cheese
         Currency: Euro
         """,
         },
         {
         "country": "Italy",
         "answer": """
         I know this:
         Capital: Rome
         Language: Italian
         Food: Pizza and Pasta
         Currency: Euro
         """,
         },
         {
         "country": "Greece",
         "answer": """
         I know this:
         Capital: Athens
         Language: Greek
         Food: Souvlaki and Feta Cheese
         Currency: Euro
         """,
    },
]


example_prompt = ChatPromptTemplate.from_messages([
    ("human","What do you know about {country}?"),
    ("ai","{answer}")
])

example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

final_prompt = ChatPromptTemplate.from_messages([
    ("system","You are a geography expert"),
    example_prompt,
    ("human","What do you know about {country}?")
])

chain = prompt | chat

chain.invoke({
    "country":"Germany"
})


AI:
         I know this:
         Capital: Berlin
         Language: German
         Food: Bratwurst and Sauerkraut
         Currency: Euro

AIMessageChunk(content='AI:\n         I know this:\n         Capital: Berlin\n         Language: German\n         Food: Bratwurst and Sauerkraut\n         Currency: Euro')

## 4.3 LengthBasedExampleSelector

In [11]:
from typing import Any, Dict, List
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.prompt import PromptTemplate
from langchain.prompts.example_selector import LengthBasedExampleSelector
from langchain.prompts.example_selector.base import BaseExampleSelector

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler()
    ]
)

examples = [
    {
        "question": "What do you know about France?",
        "answer": """ 
         Here is what I know:
         Capital: Paris
         Language: French
         Food: Wine and Cheese
         Currency: Euro
         """,
         },
         {
         "question": "What do you know about Italy?",
         "answer": """
         I know this:
         Capital: Rome
         Language: Italian
         Food: Pizza and Pasta
         Currency: Euro
         """,
         },
         {
         "question": "What do you know about Greece?",
         "answer": """
         I know this:
         Capital: Athens
         Language: Greek
         Food: Souvlaki and Feta Cheese
         Currency: Euro
         """,
    },
]

class RandomExampleSelector(BaseExampleSelector): #example들을 어떻게 선택하는지 구현
    def __init__(self, examples):
        self.examples = examples

    def add_example(self, example):
        self.examples.append(example)

    def select_examples(self, input_variables):
        from random import choice #랜덤으로 example 선택
        return [choice(self.examples)]

example_prompt = PromptTemplate.from_template("Human: {question}\nAI:{answer}")

example_selector = RandomExampleSelector( #리스트에 있는 example 중 랜덤으로 골라 가져옴
    examples = examples,
)

"""
example_selector = LengthBasedExampleSelector( #example의 양을 정할수 있음
    examples = examples,
    example_prompt=example_prompt,
    max_length=80,
)
""" 
prompt = FewShotPromptTemplate( 
    example_prompt=example_prompt,
    example_selector=example_selector,
    suffix="Human: What do you know about {country}?",
    input_variables=["country"],
)

prompt.format(country="Brazil")

'Human: What do you know about Italy?\nAI:\n         I know this:\n         Capital: Rome\n         Language: Italian\n         Food: Pizza and Pasta\n         Currency: Euro\n         \n\nHuman: What do you know about Brazil?'

## 4.4 Serialization and Composition
- 여러개의 prompt들을 하나의 prompt로 합치는 방법

In [13]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import load_prompt


#prompt = load_prompt("./prompt.json")
prompt = load_prompt("./prompt.yaml")

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

prompt.format(country="Germany")

'What is the capital of Germany'

In [21]:
from langchain.prompts.pipeline import PipelinePromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import load_prompt


#prompt = load_prompt("./prompt.json")
prompt = load_prompt("./prompt.yaml")

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

intro = PromptTemplate.from_template(
    """
    You are a role playing assistant.
    And you are impersonating a {character}
    """
)

example = PromptTemplate.from_template(
    """
    This is an example of how you talk:

    Human: {example_question}
    You: {example_answer}
    """
)

start = PromptTemplate.from_template(
    """
    Start now!

    Human: {question}
    You:
    """
)

final = PromptTemplate.from_template(
    """
    {intro}
    {example}
    {start}
    """
)

prompts = [
    ("intro", intro),
    ("example",example),
    ("start",start)
]

full_prompt = PipelinePromptTemplate( #여러개의 prompt들을 하나의 prompt로 합치는 방법
    final_prompt=final, 
    pipeline_prompts=prompts
)

#prompt.format(country="Germany")
full_prompt.format_prompt(
    character="Pirate",
    example_question="What is your location?",
    example_answer="Arrrrg! That is a secret!! Arg arg!!",
    question="What is your fav food?",
)

chain = full_prompt | chat

chain.invoke({
    "character":"Pirate",
    "example_question":"What is your location?",
    "example_answer":"Arrrrg! That is a secret!! Arg arg!!",
    "question":"What is your fav food?",
})

Arrr matey! Me favorite grub be a hearty stew made with fresh fish, potatoes, and plenty o' spices! Aye, nothing beats a good ol' bowl o' pirate stew on the high seas! Arrr!

AIMessageChunk(content="Arrr matey! Me favorite grub be a hearty stew made with fresh fish, potatoes, and plenty o' spices! Aye, nothing beats a good ol' bowl o' pirate stew on the high seas! Arrr!")

## 4.5 Caching
- 캐싱을 사용하면 LLM의 응답을 저장할 수가 있음
- 똑같은 질문을 계속 받더라도 답변을 재사용하여 비용을 절약함
- 데이터베이스에 캐싱하고 싶으면 SQLiteCache 사용

In [27]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.globals import set_llm_cache, set_debug
from langchain.cache import InMemoryCache, SQLiteCache

#set_llm_cache(InMemoryCache()) #인메모리 캐싱사용
set_llm_cache(SQLiteCache("cache.db"))
#set_debug(True)

chat = ChatOpenAI(
    temperature=0.1
)

chat.predict("How do you make italian pasta")


[llm/start] [1:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: How do you make italian pasta"
  ]
}
[llm/end] [1:llm:ChatOpenAI] [5.34s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "To make Italian pasta, you will need the following ingredients:\n\n- 2 cups of all-purpose flour\n- 2 large eggs\n- Pinch of salt\n- Water (if needed)\n\nHere is a step-by-step guide to making Italian pasta:\n\n1. On a clean work surface, pour the flour and create a well in the center.\n2. Crack the eggs into the well and add a pinch of salt.\n3. Using a fork, gradually mix the eggs into the flour until a dough forms.\n4. Knead the dough for about 10 minutes until it becomes smooth and elastic. If the dough is too dry, add a little water. If it is too wet, add a little more flour.\n5. Wrap the dough in plastic wrap and let it rest for about 30 minutes.\n6. After resting, roll out the dough using a pasta machine or a rolling pin until it is about 

'To make Italian pasta, you will need the following ingredients:\n\n- 2 cups of all-purpose flour\n- 2 large eggs\n- Pinch of salt\n- Water (if needed)\n\nHere is a step-by-step guide to making Italian pasta:\n\n1. On a clean work surface, pour the flour and create a well in the center.\n2. Crack the eggs into the well and add a pinch of salt.\n3. Using a fork, gradually mix the eggs into the flour until a dough forms.\n4. Knead the dough for about 10 minutes until it becomes smooth and elastic. If the dough is too dry, add a little water. If it is too wet, add a little more flour.\n5. Wrap the dough in plastic wrap and let it rest for about 30 minutes.\n6. After resting, roll out the dough using a pasta machine or a rolling pin until it is about 1/8 inch thick.\n7. Cut the dough into your desired shape, such as fettuccine or spaghetti.\n8. Cook the pasta in a large pot of salted boiling water for about 2-3 minutes, or until al dente.\n9. Drain the pasta and toss it with your favorite 

In [26]:
chat.predict("How do you make italian pasta") #캐싱을 사용해서 답변이 0초임

[llm/start] [1:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: How do you make italian pasta"
  ]
}
[llm/end] [1:llm:ChatOpenAI] s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "To make Italian pasta, you will need the following ingredients:\n\n- 2 cups of all-purpose flour\n- 2 large eggs\n- Pinch of salt\n- Water (if needed)\n\nHere is a step-by-step guide to making Italian pasta:\n\n1. On a clean work surface, pour the flour and make a well in the center.\n2. Crack the eggs into the well and add a pinch of salt.\n3. Using a fork, gradually mix the eggs into the flour until a dough forms.\n4. Knead the dough for about 10 minutes until it is smooth and elastic. If the dough is too dry, add a little water. If it is too wet, add a little more flour.\n5. Wrap the dough in plastic wrap and let it rest for at least 30 minutes.\n6. After resting, roll out the dough using a pasta machine or a rolling pin until it is thin.\n7. Cut t

'To make Italian pasta, you will need the following ingredients:\n\n- 2 cups of all-purpose flour\n- 2 large eggs\n- Pinch of salt\n- Water (if needed)\n\nHere is a step-by-step guide to making Italian pasta:\n\n1. On a clean work surface, pour the flour and make a well in the center.\n2. Crack the eggs into the well and add a pinch of salt.\n3. Using a fork, gradually mix the eggs into the flour until a dough forms.\n4. Knead the dough for about 10 minutes until it is smooth and elastic. If the dough is too dry, add a little water. If it is too wet, add a little more flour.\n5. Wrap the dough in plastic wrap and let it rest for at least 30 minutes.\n6. After resting, roll out the dough using a pasta machine or a rolling pin until it is thin.\n7. Cut the dough into your desired shape, such as fettuccine, spaghetti, or ravioli.\n8. Cook the pasta in a large pot of salted boiling water for 2-3 minutes or until al dente.\n9. Drain the pasta and toss it with your favorite sauce or toppings

## 4.6 Serialization
- with get_openai_callback() as usage: OpenAI 지출 비용 확인
- load_llm: 모델을 저장하고 불러오는 방법

In [2]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import get_openai_callback

chat = ChatOpenAI(
    temperature=0.1
)

with get_openai_callback() as usage:
    a = chat.predict("What is the recipe for soju")
    b = chat.predict("What is the recipe for bread")
    print(a,b,"\n")
    print(usage)

Ingredients:
- 1 cup of rice
- 1 cup of water
- 1 tablespoon of nuruk (fermentation starter)
- 1 tablespoon of yeast

Instructions:
1. Rinse the rice under cold water until the water runs clear.
2. In a large pot, combine the rice and water and bring to a boil. Reduce heat to low and simmer for 20 minutes, or until the rice is cooked.
3. Remove the pot from heat and let it cool to room temperature.
4. In a separate bowl, mix the nuruk and yeast with a little bit of warm water to form a paste.
5. Add the rice to the paste and mix well.
6. Transfer the mixture to a clean glass jar and cover with a clean cloth.
7. Let the mixture ferment at room temperature for 3-4 days, stirring occasionally.
8. After fermentation is complete, strain the mixture through a cheesecloth to remove any solids.
9. Transfer the liquid to a clean bottle and store in the refrigerator.
10. Serve chilled and enjoy your homemade soju! Ingredients:
- 3 1/2 cups all-purpose flour
- 1 packet active dry yeast
- 1 1/2 cu

In [3]:
from langchain.chat_models import ChatOpenAI
from langchain.llms.openai import OpenAI

chat = OpenAI(
    temperature=0.1,
    max_tokens=4,
    model = "gpt-3.5-turbo-16k"
)

chat.save("model.json") # 모델 저장 

In [4]:
from langchain.chat_models import ChatOpenAI
from langchain.llms.openai import OpenAI
from langchain.llms.loading import load_llm

chat = load_llm("model.json") #저장한 모델 불러오기

chat


d:\FULLSTACK-GPT\env\Lib\site-packages\langchain\llms\openai.py:216: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
d:\FULLSTACK-GPT\env\Lib\site-packages\langchain\llms\openai.py:811: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


OpenAIChat(client=<class 'openai.api_resources.chat_completion.ChatCompletion'>, model_name='gpt-3.5-turbo-16k', model_kwargs={'temperature': 0.1, 'max_tokens': 4, 'top_p': 1, 'frequency_penalty': 0, 'presence_penalty': 0, 'n': 1, 'request_timeout': None, 'logit_bias': {}})

## 5.0 ConversationBufferMemory
- OpenAI의 API는 memory가 없음. stateless 상태. 즉, 모델에게 어떤 말을 건네면 모델은 답을한직후, 대화내용을 잊어버림
- ConversationBufferMemory: 이전 대화 내용 전체를 저장하는 메모리. 대화내용이 길어질수록 메모리가 커지므로 비효율적

In [7]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(return_messages=True)

memory.save_context({"input":"Hi!"}, {"output":"How are you?"})

memory.load_memory_variables({})

{'history': [HumanMessage(content='Hi!'), AIMessage(content='How are you?')]}

## 5.1 ConversationBufferWindowMemory
- 대화 버퍼 윈도우
- k개 만큼 대화를 저장함

In [9]:
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(
    return_messages=True,
    k=4 
)

def add_message(input, output):
    memory.save_context({"input":input},{"output":output})

add_message(1,1)

In [10]:
add_message(2,2)
add_message(3,3)
add_message(4,4)

In [11]:
memory.load_memory_variables({})

{'history': [HumanMessage(content='1'),
  AIMessage(content='1'),
  HumanMessage(content='2'),
  AIMessage(content='2'),
  HumanMessage(content='3'),
  AIMessage(content='3')]}

In [12]:
add_message(5,5)

memory.load_memory_variables({})

{'history': [HumanMessage(content='1'),
  AIMessage(content='1'),
  HumanMessage(content='2'),
  AIMessage(content='2'),
  HumanMessage(content='3'),
  AIMessage(content='3'),
  HumanMessage(content='5'),
  AIMessage(content='5')]}

## 5.2 ConversationSummaryMemory

In [13]:
from langchain.memory import ConversationSummaryMemory
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0.1)

memory =  ConversationSummaryMemory(llm=llm)

def add_message(input, output):
    memory.save_context({"input":input},{"output":output})

def get_history():
    return memory.load_memory_variables({})

add_message("Hi I'm Nicolas, I live in South Korea","Wow that is so cool!")

In [ ]:
add_message("South Korea is so pretty","I with I could go!!!")

In [14]:
get_history()

{'history': 'Nicolas introduces himself as living in South Korea. The AI responds by expressing admiration for his location.'}

## 5.3 ConversationSummaryBufferMemory
- 메모리에 보내온 메시지 수 저장
= limit에 걸리면, 오래된 메시지를 제거하는 대신 요약함(history:)

In [2]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=150, #메시지 토큰수의 최대값. 메시지들이 요약되기전
    return_messages=True,
)

def add_message(input, output):
    memory.save_context({"input":input},{"output":output})

def get_history():
    return memory.load_memory_variables({})

add_message("Hi I'm Nicolas, I live in South Korea","Wow that is so cool!")


SSLError: HTTPSConnectionPool(host='openaipublic.blob.core.windows.net', port=443): Max retries exceeded with url: /encodings/cl100k_base.tiktoken (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: self-signed certificate in certificate chain (_ssl.c:1006)')))

In [ ]:
add_message("South Korea is so pretty","I with I could go!!!")

In [ ]:
get_history()

## 5.4 ConversationKGMemory(Conversation Knowledge Graph Memory)
- 대화 중의 엔티티의 knowledge graph 생성
- knowledge graph에서 히스토리를 가지고 오지 않고 엔티티를 가지고옴

In [4]:
from langchain.memory import ConversationKGMemory
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0.1)

memory = ConversationKGMemory(
    llm=llm,
    return_messages=True,
)

def add_message(input, output):
    memory.save_context({"input":input},{"output":output})

add_message("Hi I'm Nicolas, I live in South Korea","Wow that is so cool!")

memory.load_memory_variables({"input":"who is Nicolas"})

{'history': [SystemMessage(content='On Nicolas: Nicolas lives in South Korea.')]}

In [5]:
add_message("Nicolas likes kimchi","Wow that is so cool!")
memory.load_memory_variables({"input":"who is Nicolas"}) #대화에서 Entity를 뽑아냄

{'history': [SystemMessage(content='On Nicolas: Nicolas lives in South Korea. Nicolas likes kimchi.')]}

## 5.5 Memory on LLMChain
- memory를 chain에 꽂는 방법, 두종류의 chain을 사용해서 꽂는 방법
- 대화내용을 memory에 저장해두었다가, 프롬프트로 LLM에 전달한다
- LLM Chain: off-the-shelf chain. 일반적인 목적을 가진 chain.
- memory_key 변수를 통해 prompt 에 history 주입

In [27]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain #off-the-shelf chain. 여기에 갖고있는 memory 주입
from langchain.prompts import PromptTemplate

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory( #기본적으로 갖고있는 conversation class
    llm=llm,
    max_token_limit=80, #interaction 토큰수가 80개이상일경우, 가장오래된 interaction을 요약해줌
    memory_key="chat_history" #memory.load_variables 역할. 프롬프트template안에 memory가 history를 저장하도록한곳. 변수명 동일해야함
)

template = """
    You are a helpful AI talking to a human.

    {chat_history}
    Human:{question}
    You:
"""
chain = LLMChain(
    llm=llm,
    memory=memory,
    prompt=PromptTemplate.from_template(template),
    verbose=True
)

chain.predict(question="My name is Nico")

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised APIConnectionError: Error communicating with OpenAI: HTTPSConnectionPool(host='api.openai.com', port=443): Max retries exceeded with url: /v1/chat/completions (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000001927E506590>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed')).




> Entering new LLMChain chain...
Prompt after formatting:

    You are a helpful AI talking to a human.

    
    Human:My name is Nico
    You:



APIConnectionError: Error communicating with OpenAI: HTTPSConnectionPool(host='api.openai.com', port=443): Max retries exceeded with url: /v1/chat/completions (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000001927E5150D0>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))

## 5.6 Chat Based Memory
- 대화 기반 메세지의 memory
- memory를 message 형태, 문자열 형태로 출력가능
- MessagesPlaceholder: 제한없는 메세지 저장. 대화 history를 저장

In [26]:

from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain #off-the-shelf chain. 여기에 갖고있는 memory 주입
from langchain.prompts import PromptTemplate, ChatPromptTemplate, MessagesPlaceholder

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory( #기본적으로 갖고있는 conversation class
    llm=llm,
    max_token_limit=80, #interaction 토큰수가 80개이상일경우, 가장오래된 interaction을 요약해줌
    memory_key="chat_history", #memory.load_variables 역할. 프롬프트template안에 memory가 history를 저장하도록한곳. 변수명 동일해야함
    return_messages=True #실제 message클래스로 변환. ChatPromptTemplate
)

prompt = ChatPromptTemplate.from_messages([
    ("system","You are a helpful AI talking to a human"),
    MessagesPlaceholder(variable_name="chat_history"),#대화 history 저장
    ("human","{question}"),
]) #시스템 메세지 설정

chain = LLMChain(
    llm=llm,
    memory=memory,
    prompt=prompt,
    verbose=True
)

chain.predict(question="My name is Nico")

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised APIConnectionError: Error communicating with OpenAI: HTTPSConnectionPool(host='api.openai.com', port=443): Max retries exceeded with url: /v1/chat/completions (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000001927DA97790>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed')).




> Entering new LLMChain chain...
Prompt after formatting:
System: You are a helpful AI talking to a human
Human: My name is Nico


APIConnectionError: Error communicating with OpenAI: HTTPSConnectionPool(host='api.openai.com', port=443): Max retries exceeded with url: /v1/chat/completions (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000001927DA74CD0>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))

## 5.7 LCEL Based Memory
- chain.invoke에 입력한 값은 chain의 첫번째 chain으로 들어감. 아래 코드에서는 load_memory()함수의 input 파라미터로 받게됨. 파라미터가 없을경우 오류 발생.
- load_memory_variables(): 메모리를 로드시키는 함수
- save_context(): 사람과 AI의 메세지인 input,output을 메모리에 저장하는 함수

In [8]:

from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import PromptTemplate, ChatPromptTemplate, MessagesPlaceholder

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory( #기본적으로 갖고있는 conversation class
    llm=llm,
    max_token_limit=80, #interaction 토큰수가 80개이상일경우, 가장오래된 interaction을 요약해줌
    memory_key="chat_history", #memory.load_variables 역할. 프롬프트template안에 memory가 history를 저장하도록한곳. 변수명 동일해야함
    return_messages=True #실제 message클래스로 변환. ChatPromptTemplate
)

prompt = ChatPromptTemplate.from_messages([
    ("system","You are a helpful AI talking to a human"),
    MessagesPlaceholder(variable_name="chat_history"),#대화 history 저장
    ("human","{question}"),
]) #시스템 메세지 설정

""" LLMChain 대신 | 사용 
chain = LLMChain(
    llm=llm,
    memory=memory,
    prompt=prompt,
    verbose=True
)
chain.predict(question="My name is Nico")
"""

def load_memory(input):
    #print(input) input 무시
    return memory.load_memory_variables({})["chat_history"] #메모리 변수 로드

#1. 실행할 때 가장먼저 load_memory 함수 호출
#2. 프롬프트가 필요로하는 chat_history key 내부에 값저장 
chain = RunnablePassthrough.assign(chat_history=load_memory) | prompt | llm


#질문하고, Input/Output 값을 메모리에 저장
def invoke_chain(question):
    result = chain.invoke({"question": question})
    memory.save_context({"input": question}, {"output":result.content})
    print(result)


#LLM한테 질문하기!
invoke_chain("My name is nico")


SSLError: HTTPSConnectionPool(host='openaipublic.blob.core.windows.net', port=443): Max retries exceeded with url: /encodings/cl100k_base.tiktoken (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: self-signed certificate in certificate chain (_ssl.c:1006)')))

## 6.1 Data Loaders and Splitters
- 데이터 로드 > 데이터 분할(split) > Embed >  Store > Retrieve
- UnstructuredFileLoader: txt, pdf, html, image 등 로드 가능
- RecursiveCharacterTextSplitter(), CharacterTextSplitter(): Splitter 함수

In [20]:
from langchain.chat_models import ChatOpenAI
#from langchain.document_loaders import Textloader
#from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import UnstructuredFileLoader
#from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.text_splitter import CharacterTextSplitter

"""
splitter = RecursiveCharacterTextSplitter(
    chunk_size=200, #한 문단의 크기
    chunk_overlap=50 #문단을 분할할때 앞 조각 일부분 가져옴
)
"""
splitter = CharacterTextSplitter(
    separator="\n", #분할자 사용
    chunk_size=600,
    chunk_overlap=100,
)

#loader = Textloader("./files/chapter_one.txt")
#loader = PyPDFLoader("./files/chapter_one.pdf")
loader = UnstructuredFileLoader("./files/chapter_one.docx") #파일 로드

loader.load_and_split(text_splitter=splitter)
#docs = loader.load()
#splitter.split_documents(docs)




Created a chunk of size 963, which is longer than the specified 600
Created a chunk of size 774, which is longer than the specified 600
Created a chunk of size 954, which is longer than the specified 600
Created a chunk of size 922, which is longer than the specified 600
Created a chunk of size 1168, which is longer than the specified 600
Created a chunk of size 821, which is longer than the specified 600
Created a chunk of size 700, which is longer than the specified 600
Created a chunk of size 745, which is longer than the specified 600
Created a chunk of size 735, which is longer than the specified 600
Created a chunk of size 1110, which is longer than the specified 600
Created a chunk of size 991, which is longer than the specified 600
Created a chunk of size 990, which is longer than the specified 600
Created a chunk of size 1741, which is longer than the specified 600
Created a chunk of size 2001, which is longer than the specified 600
Created a chunk of size 1900, which is longe

[Document(page_content='Part One\n1\nIt was a bright cold day in April, and the clocks were striking thirteen. Winston Smith, his chin nuzzled into his breast in an effort to escape the vile wind, slipped quickly through the glass doors of Victory Mansions, though not quickly enough to prevent a swirl of gritty dust from entering along with him.', metadata={'source': './files/chapter_one.docx'}),
 Document(page_content='The hallway smelt of boiled cabbage and old rag mats. At one end of it a coloured poster, too large for indoor display, had been tacked to the wall. It depicted simply an enormous face, more than a metre wide: the face of a man of about forty-five, with a heavy black moustache and ruggedly handsome features. Winston made for the stairs. It was no use trying the lift. Even at the best of times it was seldom working, and at present the electric current was cut off during daylight hours. It was part of the economy drive in preparation for Hate Week. The flat was seven flig

## 6.2 Tiktoken
- length_function: splitter가 텍스트 length 계산하여 한 chunk 크기를 알아냄. default는 표준 라이브러리 len() 함수.
- model의 관점에서 몇개의 token을 사용하는 지 계산 
[https://platform.openai.com/tokenizer](https://platform.openai.com/tokenizer)
- model은 token간의 통계적인 관계를 이해하도록 학습을 함
- 예를 들어, app다음에 le가 오는걸 알수도 있음

In [21]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n", #분할자 사용
    chunk_size=600,
    chunk_overlap=100,
    #length_function=len,
)

loader = UnstructuredFileLoader("./files/chapter_one.docx") #파일 로드

loader.load_and_split(text_splitter=splitter)

SSLError: HTTPSConnectionPool(host='openaipublic.blob.core.windows.net', port=443): Max retries exceeded with url: /gpt-2/encodings/main/vocab.bpe (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: self-signed certificate in certificate chain (_ssl.c:1006)')))

## 6.3 Vectors
- Embedding: 사람이 읽는 텍스트를 컴퓨터가 이해할수있는 숫자들로 변환하는 작업
- Vectorization(벡터화): 문서마다 각각의 벡터 생성

## 6.4 Vector Store
- split했었던 문서에 embed 작업
- embedding을 vector store에 캐싱
= OpenAIEmbeddings: OpenAI의 embeddings은 text-embedding-ada-002 model 사용.

In [24]:
from langchain.embeddings import OpenAIEmbeddings

embedder = OpenAIEmbeddings()

#embedder.embed_query("HI") #Hi에 해당하는 벡터 출력

vector = embedder.embed_documents(
    "hi",
    "how",
    "are",
    "you"
)


TypeError: OpenAIEmbeddings.embed_documents() takes from 2 to 3 positional arguments but 5 were given

In [28]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import Chroma
from langchain.storage import LocalFileStore

cache_dir = LocalFileStore("./.cache/") #cache directory

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n", #분할자 사용
    chunk_size=600,
    chunk_overlap=100
)

loader = UnstructuredFileLoader("./files/chapter_one.docx") #파일 로드

docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()

#embedding 작업을 위해 필요한 embedder의 입력
#파일 임베딩 및 벡터스토어 저장
cached_embeddings = CacheBackedEmbeddings.from_bytes_store(
    embeddings, cache_dir
)

#1. 캐시에 embeddings가 이미 존재하는지 확인
#2. 만약없다면, vector store(Chroma.from_documents)를 호출할 때, 문서들과(docs) 함께 OpenAIEmbeddings 사용
vectorstore = Chroma.from_documents(docs, cached_embeddings)

results = vectorstore.similarity_search("where does winston live") #벡터스토어 활용하여 검색
#질문과 관련있는 문서를 results로 반환. results를 통채로 LLM에 넘기면됨


## 6.6 RetrievalQA
- off-the-shelf chain 사용
- LLMChain 보다 LCEL 사용 권장
- retriever: document를 많은 장소로부터 Retrieve(선별하여 가져오기) 가능. db, cloud, vector store 등 에서 검색 가능함. stuff, refine 등 종류가 있음
- Stuff documents: 찾은 document들로 prompt를 stuff(채우기)하는데 사용
- Refine documents chain: 각각 document를 읽으면서, 질문에 대한 답변 생성을 시도함. 반복하면서 모든 document를 통해 답변을 개선시킴. 요금이 비쌈
- Map reduce documents chain: document를 입력받아서, 개별적으로 요약작업을 수행함. 
- Map re-rank documents chain: 각 document를 통해 답변을 생성하고, 각 답변에 점수를 매김. 최종적으로는 가장 높은 점수를 획득한 답변과 그 점수를 함께 반환해줌.


In [32]:
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS #파이스
from langchain.storage import LocalFileStore
from langchain.chains import RetrievalQA

llm = ChatOpenAI()

cache_dir = LocalFileStore("./.cache/") #cache directory

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n", #분할자 사용
    chunk_size=600,
    chunk_overlap=100
)

loader = UnstructuredFileLoader("./files/chapter_one.docx") #파일 로드

docs = loader.load_and_split(text_splitter=splitter)
embeddings = OpenAIEmbeddings()

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)

vectorstore = FAISS.from_documents(docs, cached_embeddings)

chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="refine", #map_reduce, map_rerank
    retriever=vectorstore.as_retriever(),
)

chain.run("Where does Winston live?")

"The new context provided does not directly relate to Winston's living situation in Victory Mansions. Therefore, the original answer remains appropriate. Winston lives in Victory Mansions, a run-down building in a society dominated by surveillance and control."

## 6.8 Stuff LCEL Chain
- prompt 와 retrieve 함께 사용하는 chain 만드는 법

In [ ]:
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS #파이스
from langchain.storage import LocalFileStore
from langchain.chains import RetrievalQA
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough

llm = ChatOpenAI(
    temperature=0.1
)

cache_dir = LocalFileStore("./.cache/") #cache directory

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n", #분할자 사용
    chunk_size=600,
    chunk_overlap=100
)

loader = UnstructuredFileLoader("./files/chapter_one.docx") #파일 로드

docs = loader.load_and_split(text_splitter=splitter)
embeddings = OpenAIEmbeddings()

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)

vectorstore = FAISS.from_documents(docs, cached_embeddings)

retriver = vectorstore.as_retriever()

prompt = ChatPromptTemplate.from_messages([
    ("system","You are a helpful assistant. Answer question using only the following context. If you don't know the answer just say you don't know, don't make it up:\n\n{context}"),
    ("human","{question}")
])

chain = ({
    "context": retriver,
    "question": RunnablePassthrough(),
    } 
    | prompt 
    | llm
)

chain.invoke("Where does Winston live?")

## 6.9 Map Reduce LCEL Chain
- RunnableLambda: chain과 그 내부 어디에서든 function을 호출할수 있도록 함

In [37]:
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS #파이스
from langchain.storage import LocalFileStore
from langchain.chains import RetrievalQA
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda

llm = ChatOpenAI(
    temperature=0.1
)

cache_dir = LocalFileStore("./.cache/") #cache directory

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n", #분할자 사용
    chunk_size=600,
    chunk_overlap=100
)

loader = UnstructuredFileLoader("./files/chapter_one.docx") #파일 로드

docs = loader.load_and_split(text_splitter=splitter)
embeddings = OpenAIEmbeddings()

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)

vectorstore = FAISS.from_documents(docs, cached_embeddings)

retriver = vectorstore.as_retriever()

#1. list of docs
#2. for doc in list of docs | prompt | llm
#3. for response in list of llms response | put them all together
map_doc_prompt = ChatPromptTemplate.from_messages(
    [
        ("system",
         """
         Use the following portion of a long document to see if any of the text is relevant to answer the question. Return any relevant text verbatim.
         ------
         {context}
         """),
        ("human","{question}")
    ]
)
map_doc_chain = map_doc_prompt | llm


def map_docs(inputs):
    documents = inputs["documents"]
    question = inputs["question"]
    return "\n\n".join(
        map_doc_chain.invoke(
            {"context": doc.page_content,"question": question}
        ).content
        for doc in documents
    ) #하나의 string으로 반환

map_chain = {
    "documents": retriver, 
    "question": RunnablePassthrough()
} | RunnableLambda(map_docs)

final_prompt = ChatPromptTemplate.from_messages([
    ("system","You are a helpful assistant. Answer question using only the following context. If you don't know the answer just say you don't know, don't make it up:\n\n{context}"),
    ("human","{question}")
])

#4. final doc | prompt | llm
chain = {"context": map_chain,"question": RunnablePassthrough()} | final_prompt | llm

chain.invoke("Where does Winston live?")

AIMessage(content='Winston lives in Victory Mansions on the seventh floor, specifically in a flat with a telescreen in the living-room.')